In [ ]:
import os
!pip install google-cloud-speech
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/drive/My Drive/My First Project-8084512d8249.json' #키값 설정

In [ ]:

#storage에서 오디오 파일 불러오는 방식
#stroage에 버킷 있는지 확인
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)
implicit()

[<Bucket: prime_brc>, <Bucket: stt_hana>]


In [ ]:


from google.cloud import speech_v1
from google.cloud.speech_v1 import enums

def sample_long_running_recognize(storage_uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.raw'

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 16000

    # The language of the supplied audio
    language_code = "ko-KR"

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "encoding": encoding,
    }
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
    return response

response = sample_long_running_recognize("gs://prime_brc/hana_audio.wav") #버킷명 수정


In [ ]:
with open('/content/drive/My Drive/hana_scripts.txt', "w") as script: #파일명 설정, 저장
  for result in response.results:
    script.write(u'{}'.format(result.alternatives[0].transcript)+'\n')
print('completed')  

completed


In [ ]:

from google.cloud import speech_v1
from google.cloud.speech_v1 import enums

def sample_long_running_recognize_timestamps(storage_uri):
    """
    Print start and end time of each word spoken in audio file from Cloud Storage

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.flac'

    # When enabled, the first result returned by the API will include a list
    # of words and the start and end time offsets (timestamps) for those words.
    enable_word_time_offsets = True
    sample_rate_hertz = 16000
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    # The language of the supplied audio
    language_code = "ko-KR"
    config = {
        "enable_word_time_offsets": enable_word_time_offsets, #timestamp 저장하는 설정
        "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "encoding": encoding,
    }
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    # The first result includes start and end time word offsets
    result = response.results[0]
    # First alternative is the most probable result
    alternative = result.alternatives[0]
    print(u"Transcript: {}".format(alternative.transcript))
    # Print the start and end time of each word
    for word in alternative.words:
        #타임스탬프 형식 설정
        print(u"Word: {}".format(word.word))
        print(
            u"Start time: {} seconds {} nanos".format(
                word.start_time.seconds, word.start_time.nanos
            )
        )
        print(
            u"End time: {} seconds {} nanos".format(
                word.end_time.seconds, word.end_time.nanos
            )
        )
        
    return response 
response_timestamps = sample_long_running_recognize_timestamps("gs://prime_brc/hana_audio.wav") #파일 불러오기

Waiting for operation to complete...
Transcript: 5시
Word: 5시
Start time: 50 seconds 700000000 nanos
End time: 50 seconds 700000000 nanos


In [ ]:

with open('/content/drive/My Drive/hanascripts_timestamps_ver2.txt', "w") as script: # 파일명 설정
  for result_ts in response_timestamps.results:
    for word_ts in result_ts.alternatives[0].words:
        script.write(u'{}, {}, {}'.format(word_ts.word, word_ts.start_time.seconds, word_ts.start_time.nanos )+'\n')
print('completed')

completed
